<a href="https://colab.research.google.com/github/andheartsjaz/Data-Science-Thinkful-PairProgramming-Lectures/blob/master/Support_Vector_Machines_Day_34_Lecture_1_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Day 34 Lecture 1 Assignment

In this assignment, we will learn about SVR. We will use the absenteeism at work dataset loaded below and analyze the model generated for this dataset.

In [0]:
%matplotlib inline

import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVR

In [0]:
df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Absenteeism_at_work.csv', sep=';')

In [0]:
df.head()

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,...,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,4
1,36,0,7,3,1,118,13,18,50,239.554,...,1,1,1,1,0,0,98,178,31,0
2,3,23,7,4,1,179,51,18,38,239.554,...,0,1,0,1,0,0,89,170,31,2
3,7,7,7,5,1,279,5,14,39,239.554,...,0,1,2,1,1,0,68,168,24,4
4,11,23,7,5,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,2


Recall that we removed all correlated columns as well as columns that are a function of two or more other columns (like BMI and weight and height) as well as columns that are not usefu to us (like ID).
We also created dummy variables for the categorical columns in our dataset.
Finally, we split the data into test and train (with test being 20% of the data).

Perform these actions below.

In [0]:
#answer below:

df.head()
# pd.DataFrame(df.isnull().sum() * 100 / df.shape[0], columns=["Missing Values %"])
missing = pd.DataFrame(df.isnull().sum() * 100 / len(df), columns=["Missing Values %"])
missing = missing.sort_values(by="Missing Values %", ascending=False)

# df = absent
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("Top Absolute Correlations")
print(get_top_abs_correlations(df, 5))


Top Absolute Correlations
Weight              Body mass index                    0.904117
Service time        Age                                0.670979
Reason for absence  Disciplinary failure               0.545054
Service time        Body mass index                    0.499718
ID                  Distance from Residence to Work    0.486160
dtype: float64


In [0]:
df.columns

Index(['ID', 'Reason for absence', 'Month of absence', 'Day of the week',
       'Seasons', 'Transportation expense', 'Distance from Residence to Work',
       'Service time', 'Age', 'Work load Average/day ', 'Hit target',
       'Disciplinary failure', 'Education', 'Son', 'Social drinker',
       'Social smoker', 'Pet', 'Weight', 'Height', 'Body mass index',
       'Absenteeism time in hours'],
      dtype='object')

In [0]:
# drop Weight and Height

df.drop(columns=['Weight', 'Height', 'ID', 'Seasons','Service time'])

cat_cols = ['Reason for absence', 'Month of absence', 'Day of the week', 'Hit target', 'Son', 'Pet' ]

dummy_df = pd.get_dummies(data=df, columns=cat_cols, drop_first=True, dtype=int)

Scale the indepdendent variables using the standard scaler

In [0]:
s = StandardScaler()
# s = MinMaxScaler()
# s = None

In [0]:
X = dummy_df.drop(columns='Absenteeism time in hours')
y = dummy_df['Absenteeism time in hours']


In [0]:
# answer below:

if s is not None:
    s.fit_transform(X_train)

    X_train = s.transform(X_train)
    X_test = s.transform(X_test)

C:\Users\jvicc\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\jvicc\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\jvicc\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  
C:\Users\jvicc\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  import sys


Split the data into train and test (test proportion=0.2).

In [0]:
# answer below:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                        test_size=0.2, 
#                                         stratify=y, 
                                        random_state=42) # don't stratify by 'y' in a Regression problem

Generate an SVR model to predict the number of hours of absenteeism. Compare the train and test scores.

In [0]:
# answer below:
svm = SVR()
svm.fit(X_train, y_train)

train_score = svm.score(X_train, y_train) # returns R^2
test_score = svm.score(X_test, y_test)

print(f'Train R^2: {train_score}')
print(f'Test R^2: {test_score}')



C:\Users\jvicc\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Train R^2: 0.07605936100384193
Test R^2: 0.07767824623212982


In [0]:
params = {'kernel': ['linear', 'poly', 'rbf'],
          'C': [100, 1000, 10000],
          'degree': [3, 5, 5],
          'epsilon': [0.1, 1, 10]}

svm = SVR(gamma='scale')
svm_cv = GridSearchCV(svm, params, cv=2, verbose=1)
svm_cv.fit(X_train, y_train)

train_score = svm_cv.score(X_train, y_train)
test_score = svm_cv.score(X_test, y_test)

print(f'Train R2: {train_score}')
print(f'Test R2: {test_score}\n')

svm_cv.best_params_

y_pred = svm_cv.predict(X_test)

In [0]:
from sklearn.metrics import mean_absolute_error

y_pred = svm_cv.predict(X_test)

mae = abs(y_test - y_pred).mean()
mape = (abs(y_test - y_pred) / y_test).mean()
print(f'Mean Absolute Error: {mae}')
print(f'Mean Absolute Percent Error: {100 * mape}')

plt.scatter(y_pred, y_test)
plt.plot([1, 60000], [1, 60000], c='black', alpha=0.5)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

If you are unhappy with your model's performance, perform a grid search to optimize C.

In [0]:
# answer below:

